In [2]:
import pandas as pd

messages = pd.read_csv('SMSSpamCollection.txt', sep='\t',
                           names=["label", "message"])

In [3]:
messages.head()

,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [43]:
messages.isnull().sum()

label      0
message    0
dtype: int64

In [46]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(messages['message'], messages['label'], test_size=0.2, random_state=42)

In [47]:
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

(4457,) (1115,) (4457,) (1115,)


In [4]:
from nltk.stem import WordNetLemmatizer
lemmatizer=WordNetLemmatizer()

In [51]:
import re
from nltk.corpus import stopwords

corpus = []
for i in x_train:
    review = re.sub('[^a-zA-Z]', ' ', i)
    review = review.lower()
    review = review.split()
    
    review = [lemmatizer.lemmatize(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [9]:
%pip install gensim

   ---------------------------------------- 0.0/24.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/24.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/24.0 MB ? eta -:--:--
   ---------------------------------------- 0.1/24.0 MB 504.4 kB/s eta 0:00:48
   ---------------------------------------- 0.1/24.0 MB 504.4 kB/s eta 0:00:48
   ---------------------------------------- 0.2/24.0 MB 437.1 kB/s eta 0:00:55
   ---------------------------------------- 0.2/24.0 MB 437.1 kB/s eta 0:00:55
   ---------------------------------------- 0.2/24.0 MB 378.3 kB/s eta 0:01:04
   ---------------------------------------- 0.2/24.0 MB 393.0 kB/s eta 0:01:01
   ---------------------------------------- 0.2/24.0 MB 393.0 kB/s eta 0:01:01
   ---------------------------------------- 0.2/24.0 MB 393.0 kB/s eta 0:01:01
   ---------------------------------------- 0.2/24.0 MB 344.8 kB/s eta 0:01:10
   ---------------------------------------- 0.2/24.0 MB 344.8 kB/s eta 0:01:10


In [10]:
from nltk import sent_tokenize
import gensim
from gensim.utils import simple_preprocess

In [52]:
corpus[0]

'reply win weekly fifa world cup held send stop end service'

In [53]:
len(corpus)

4457

In [54]:
words=[]
for sent in corpus:
    sent_token=sent_tokenize(sent)
    for sent in sent_token:
        words.append(simple_preprocess(sent))

In [55]:
words[:2]

[['reply',
  'win',
  'weekly',
  'fifa',
  'world',
  'cup',
  'held',
  'send',
  'stop',
  'end',
  'service'],
 ['hello',
  'sort',
  'town',
  'already',
  'dont',
  'rush',
  'home',
  'eating',
  'nacho',
  'let',
  'know',
  'eta']]

In [56]:
len(words)

4450

In [57]:
print(corpus[-1])
print(words[-1])

say somebody named tampa
['say', 'somebody', 'named', 'tampa']


In [58]:
model = gensim.models.Word2Vec(words, window=5)

In [59]:
model.corpus_count

4450

In [18]:
import numpy as np

def avgWord2Vec(doc):
    return np.mean([model.wv[word] for word in doc if word in model.wv.index_to_key], axis=0)

In [19]:
%pip install tqdm

Note: you may need to restart the kernel to use updated packages.


In [20]:
from tqdm import tqdm

In [60]:
X = []
for i in tqdm(range(len(words))):
    X.append(avgWord2Vec(words[i]))

  0%|          | 0/4450 [00:00<?, ?it/s]c:\Users\karat\anaconda3\Lib\site-packages\numpy\core\fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\karat\anaconda3\Lib\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
100%|██████████| 4450/4450 [00:01<00:00, 3653.79it/s]


In [61]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

forest = RandomForestClassifier(n_estimators=100, random_state=42)

In [62]:
len(X)

4450

In [63]:
len(y_train)

4457

In [64]:
forest.fit(X, y_train)

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (4450,) + inhomogeneous part.